# Computer Vision Coursework Submission (IN3060/INM460)

**Student name, ID and cohort:** John Doe (200056789) - UG


# Notebook Setup
In this section you should include all the code cells required to test your coursework submission. Specifically:

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Define Local Path

In the next cell you should assign to the variable `GOOGLE_DRIVE_PATH_AFTER_MYDRIVE` the relative path of this folder in your Google Drive.

**IMPORTANT:** you have to make sure that **all the files required to test your functions are loaded using this variable** (as was the case for all lab tutorials). In other words, do not use in the notebook any absolute paths. This will ensure that the markers can run your functions. Also, **do not use** the magic command `%cd` to change directory.



In [ ]:
import os

# TODO: Fill in the Google Drive path where you uploaded the CW_folder_UG
# Example: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Computer Vision/CW_folder_UG'

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/CW_Folder_UG'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['.DS_Store', 'test_functions.ipynb', 'Personal_Dataset', 'Models', 'CW_Dataset', 'Code']


### Load packages 

In the next cell you should load all the packages required to test your functions.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.fft import fft2, ifft2
from skimage import color
from skimage.transform import resize
from skimage.feature import hog

import skimage.feature
import skimage.data as data
from skimage.transform import integral_image

from scipy import ndimage, misc

from scipy.signal import gaussian, convolve2d,wiener
from joblib import dump, load
from PIL import Image
import random
import pandas as pd
import os
import tensorflow as tf
import cv2
import tensorflow 
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

skip = True

# Loading in images test
I followed this youtube tutorial to get an idea of how I wanted to implement importing the images https://www.youtube.com/watch?v=j-3vuBynnOE 

In [ ]:
#reference https://www.youtube.com/watch?v=j-3vuBynnOE 
#Method 1 
if not skip:
  plt.figure(figsize=(20,20))
  IMAGE_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset/train')
  for i in range(5):
      file = random.choice(os.listdir(IMAGE_FOLDER))
      image_path= os.path.join(IMAGE_FOLDER, file)
      img=mpimg.imread(image_path)
      ax=plt.subplot(1,5,i+1)
      ax.title.set_text(file)
      plt.imshow(img)

In [ ]:
#Method 2
if not skip:
  def randomTraining(numberOfSamples):
      samples = random.sample(range(0, 12271), numberOfSamples)
      fileNames = [] 
      for samp in samples:
        leading0s = "0" * (5 - len(str(samp)))
        fileNames.append("train_"+leading0s+str(samp)+"_aligned.jpg")
      return samples, fileNames

  def showSamples(numbers,paths):
    fig, ax = plt.subplots(1, len(paths), figsize=(20, 25))

    for count,path in enumerate(paths):
      image_path= os.path.join(IMAGE_FOLDER, path)
      img = mpimg.imread(image_path)
      ax[count].imshow(img)
      ax[count].set_title(numbers[count]), ax[0].axis('off')

    fig.tight_layout()
    plt.show()

  def getSamples(paths):
    images = []
    for path in paths:
      image_path= os.path.join(IMAGE_FOLDER, path)
      img = mpimg.imread(image_path)
      images.append(img) 
    return images 
      

    

  sampleNumbers, sampleNames = randomTraining(10)
  showSamples(sampleNumbers,sampleNames)


In [ ]:
#Method 3 
if not skip:
  def getSamplesPIL(paths):
    images = []
    for path in paths:
      image_path= os.path.join(IMAGE_FOLDER, path)
      img = Image.open(image_path)
      imgData = np.asarray(img)
      images.append(imgData)
    images = np.array(images)
    return images

  def showSamples(numbers,data,grey = False ):
    fig, ax = plt.subplots(1, 10, figsize=(20, 25))
    for count,image in enumerate(data):
      if(grey):
        ax[count].imshow(image,cmap="gray")
      else:
        ax[count].imshow(image)


      ax[count].set_title(numbers[count]), ax[0].axis('off')

    fig.tight_layout()
    plt.show()

  sampleNumbers, sampleNames = randomTraining(10)

  sampleData = getSamplesPIL(sampleNames)
  showSamples(sampleNumbers,sampleData)


**Applying Smoothing Filters**

<break>The first step of my image processing is to convert to greyscale then apply median, wiener and guassian filters. After choosing one based on the compared PSNR and RSME values. 



<break> I found these methods in a research paper here https://www.ijeat.org/wp-content/uploads/papers/v6i6/F5131086617.pdf 
<break>


Wiener smoothing implementaton comes from this article https://www.researchgate.net/figure/Wiener-Filter-implementation-using-Python_fig3_332574579 This includes the Guassian kernal and wiener filter functions 
<break>


Median filter implementation found here https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.median_filter.html 
<break> 


Gaussian blue implementation found here https://www.tutorialkart.com/opencv/python/opencv-python-gaussian-image-smoothing/  

In [ ]:
if not skip: 
  #Convert to greyscale
  graySamplesTest = np.empty((len(sampleData),100,100))
  medianSamples = np.empty((len(sampleData),100,100))
  wienerSamples = np.empty((len(sampleData),100,100))
  gaussianSamples = np.empty((len(sampleData),100,100))

  smoothFactor = 2


  print(graySamplesTest.shape)

  print("Grayscale Images")
  for count,image in enumerate(sampleData):
    grayImage = color.rgb2gray(image) 
    graySamplesTest[count] = grayImage

  showSamples(sampleNumbers,graySamplesTest,True)

  print("Median Images")
  #creating the median filter images
  #https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.median_filter.html 
  for count,image in enumerate(graySamplesTest):
    medianImage = ndimage.median_filter(image, size=smoothFactor)
    medianSamples[count] = medianImage

  showSamples(sampleNumbers,medianSamples,True)

#https://www.researchgate.net/figure/Wiener-Filter-implementation-using-Python_fig3_332574579
  def gaussian_kernal(kernel_size =3):
    h = gaussian(kernel_size,kernel_size/3).reshape(kernel_size,1) 
    h = np.dot(h,h.transpose())
    h /= np.sum(h) 
    return h 

  def wiener_filter(img,kernel,k):
    kernel /= np.sum(kernel)
    dummy = np.copy(img)
    dummy = fft2(dummy)
    kernel = fft2(kernel, s=img.shape)
    kernel = np.conj(kernel) / (np.abs(kernel)**2 + k)
    dummy = dummy * kernel
    dummy = np.abs(ifft2(dummy))
    return dummy 

  kernel = gaussian_kernal(smoothFactor)


  print("Wiener Images")
  #creating the median filter images
  for count,image in enumerate(graySamplesTest):
    wienerImage = wiener_filter(image,kernel,k=5)
    wienerSamples[count] = wienerImage

  showSamples(sampleNumbers,wienerSamples,True)


#reference https://www.tutorialkart.com/opencv/python/opencv-python-gaussian-image-smoothing/  
  print("Gaussian Images")
  #creating the Gaussian filter images
  for count,image in enumerate(graySamplesTest):
    gaussianImage = cv2.GaussianBlur(image,(5,5),cv2.BORDER_DEFAULT) 
    gaussianSamples[count] = gaussianImage

  showSamples(sampleNumbers,gaussianSamples,True)



**Calculate the PSNR and RMSE**
<break> Calculating this for each filter will allow us to see which one should be used for the image

To calculate PSNR I followed this https://www.geeksforgeeks.org/python-peak-signal-to-noise-ratio-psnr/ 

To calculate RMSE I followed this https://www.codegrepper.com/code-examples/python/how+to+find+MSE+of+an+image+in+python 

In [ ]:
if not skip:
  #reference https://www.geeksforgeeks.org/python-peak-signal-to-noise-ratio-psnr/ 

  def PSNR(original, smoothened):
      mse = np.mean((original - smoothened) ** 2)
      if(mse == 0):  # MSE is zero means no noise is present in the signal .
                    # Therefore PSNR have no importance.
          return 100
      max_pixel = 255.0
      psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
      return psnr
#reference https://www.codegrepper.com/code-examples/python/how+to+find+MSE+of+an+image+in+python  
  def RMSE(original, smoothened):
      return np.sqrt(((original - smoothened) ** 2).mean())
    
  for count,image in enumerate(graySamplesTest):
      #median
      medPSNR = PSNR(image,medianSamples[count])
      medRMSE = RMSE(image,medianSamples[count])
      print("Median Filter Values")
      print("PSNR: ",medPSNR)
      print("RMSE: ",medRMSE)

      #Wiener
      wienerPSNR = PSNR(image,wienerSamples[count])
      wienerRMSE = RMSE(image,wienerSamples[count])
      print("Wiener Filter Values")
      print("PSNR: ",wienerPSNR)
      print("RMSE: ",wienerRMSE)

      #Gaussian
      gaussianPSNR = PSNR(image,gaussianSamples[count])
      gaussianRMSE = RMSE(image,gaussianSamples[count])
      print("Gaussian Filter Values")
      print("PSNR: ",gaussianPSNR)
      print("RMSE: ",gaussianRMSE,"\n")





# Thoughts on the smoothing filters
The paper I followed showed how different filters can be used better on different images however I ran a lot of tests using the code above and found the Wiener filter always has the best values so I will use that for the first stage of my pipeline. 

# Feature Descriptors
I will be running a combined set of descriptors of both HOG and LBP as a research paper I found showed that they make a very good mix. First I will text them individually then get the combined feature vector.

**References for the code below**

Implementation of HOG https://www.thepythoncode.com/article/hog-feature-extraction-in-python https://ieeexplore.ieee.org/document/5771368 

Implementation of LBP https://fairyonice.github.io/implement-lbp-from%20scratch.html https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_local_binary_pattern.html

Wiener Filter smoothing https://www.researchgate.net/figure/Wiener-Filter-implementation-using-Python_fig3_332574579 

Exposure balancing implementation I followed https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_histograms/py_histogram_equalization/py_histogram_equalization.html 

In [1]:
if not skip:
	##reference https://www.thepythoncode.com/article/hog-feature-extraction-in-python https://ieeexplore.ieee.org/document/5771368 

	#HOG Feature Descriptor
	grayimg = cv2.cvtColor(sampleData[1], cv2.COLOR_BGR2GRAY)
	equalisedImage = cv2.equalizeHist(grayimg)

	fd, hog_image = hog(resize(equalisedImage, (128*4, 64*4)), orientations=9, pixels_per_cell=(8, 8),
										cells_per_block=(2, 2), visualize=True, multichannel=False) 
	plt.axis("off")
	plt.imshow(hog_image, cmap="gray")
	print(fd)
	print(fd.shape)

NameError: ignored

In [ ]:
#LBP Method 1
if not skip:
  # Source of this function 
  #https://fairyonice.github.io/implement-lbp-from%20scratch.html 
  def getLBPimage(gray_image):
      ### Step 0: Step 0: Convert an image to grayscale
      imgLBP = np.zeros_like(gray_image)
      neighboor = 3 
      for ih in range(0,image.shape[0] - neighboor):
          for iw in range(0,image.shape[1] - neighboor):
              ### Step 1: 3 by 3 pixel
              img          = gray_image[ih:ih+neighboor,iw:iw+neighboor]
              center       = img[1,1]
              img01        = (img >= center)*1.0
              img01_vector = img01.T.flatten()
              # it is ok to order counterclock manner
              # img01_vector = img01.flatten()
              ### Step 2: **Binary operation**:
              img01_vector = np.delete(img01_vector,4)
              ### Step 3: Decimal: Convert the binary operated values to a digit.
              where_img01_vector = np.where(img01_vector)[0]
              if len(where_img01_vector) >= 1:
                  num = np.sum(2**where_img01_vector)
              else:
                  num = 0
              imgLBP[ih+1,iw+1] = num
      return(imgLBP)

  grayimg = cv2.cvtColor(sampleData[0], cv2.COLOR_BGR2GRAY)
  equalisedImage = cv2.equalizeHist(grayimg)

  lbpImage = getLBPimage(equalisedImage)

  plt.axis("off")
  plt.imshow(lbpImage, cmap="gray")
  print(lbpImage.shape)
  flatLBP = lbpImage.flatten() 
  print(flatLBP.shape)

**Combination of HOG and LBP**
With the two feature descriptors working independantly we can take the two feature vectors and concatenate them together, this will then be the input for our classifier.This will combine everything above including the wiener smoothing and auto exposure. 

In [ ]:
#Implementation draft 1, everything needed will be in this cell block 
def gaussian_kernal(kernel_size =3):
  h = gaussian(kernel_size,kernel_size/3).reshape(kernel_size,1) 
  h = np.dot(h,h.transpose())
  h /= np.sum(h) 
  return h 

def wiener_filter(img,kernel,k):
  kernel /= np.sum(kernel)
  dummy = np.copy(img)
  dummy = fft2(dummy)
  kernel = fft2(kernel, s=img.shape)
  kernel = np.conj(kernel) / (np.abs(kernel)**2 + k)
  dummy = dummy * kernel
  dummy = np.abs(ifft2(dummy))
  return dummy 

def convert_space(img):
  for count, pixel in enumerate(img):
    print(pixel)
   # img[count] = (int)(pixel*255.999)
  return img

def getLBPimage(gray_image):
    ### Step 0: Step 0: Convert an image to grayscale
    imgLBP = np.zeros_like(gray_image)
    neighboor = 3 
    for ih in range(0,image.shape[0] - neighboor):
        for iw in range(0,image.shape[1] - neighboor):
            ### Step 1: 3 by 3 pixel
            img          = gray_image[ih:ih+neighboor,iw:iw+neighboor]
            center       = img[1,1]
            img01        = (img >= center)*1.0
            img01_vector = img01.T.flatten()
            # it is ok to order counterclock manner
            # img01_vector = img01.flatten()
            ### Step 2: **Binary operation**:
            img01_vector = np.delete(img01_vector,4)
            ### Step 3: Decimal: Convert the binary operated values to a digit.
            where_img01_vector = np.where(img01_vector)[0]
            if len(where_img01_vector) >= 1:
                num = np.sum(2**where_img01_vector)
            else:
                num = 0
            imgLBP[ih+1,iw+1] = num
    return(imgLBP)


def get_feature_vector(img):
  #Equalise the image exposure
  equalisedImage = cv2.equalizeHist(img)

  #Smooth noise with Wiener
  kernel = gaussian_kernal(4)
  wienerImage = wiener_filter(equalisedImage,kernel,k=5)

  #Get LBP feature vector
  lbpImage = getLBPimage(wienerImage)

  lbpVector = lbpImage.flatten() 

  #Get HOG feature vector

  #print("LBP Size", lbpVector.shape)
 # hogVector = hog(resize(wienerImage, (128*4, 64*4)), orientations=9, pixels_per_cell=(8, 8),
              # 	cells_per_block=(2, 2), visualize=False, multichannel=False) 
  
  hogVector = hog(wienerImage, orientations=9, pixels_per_cell=(5,5),cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False,  transform_sqrt=False, feature_vector=True, multichannel=None)
  
 # print("Hog Size", hogVector.shape)
  
 # plt.imshow(wienerImage,cmap="gray")

  featureVector = np.concatenate((hogVector,lbpVector))
  return featureVector


#feature_vector = get_feature_vector(sampleData[0])

#print(feature_vector)
#print(feature_vector.shape)
#print(feature_vector.size)


# Processing Lables
The next logical step is to handle the labels. Training and classification can't be done without handling lables 

In [ ]:
LABEL_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset/labels')


def get_train_labels():
  with open(os.path.join(LABEL_FOLDER,'list_label_train.txt'),'r') as f:
    labels = f.readlines()
  f.close()
  labels = [line[16:] for line in labels]
  labels = [line[:1] for line in labels]
  labels = list(map(int, labels))
  return labels

train_labels = get_train_labels()

print(train_labels)

print(train_labels[400 -1])


[5, 5, 4, 4, 5, 1, 5, 4, 4, 1, 4, 1, 1, 4, 5, 4, 4, 4, 4, 4, 4, 4, 6, 5, 5, 5, 2, 5, 4, 5, 3, 1, 5, 1, 4, 4, 4, 5, 5, 5, 5, 5, 1, 1, 4, 4, 6, 4, 6, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 1, 3, 6, 4, 5, 4, 1, 1, 6, 3, 1, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 5, 4, 4, 1, 4, 4, 4, 6, 4, 5, 4, 4, 4, 5, 5, 3, 1, 6, 1, 1, 4, 5, 1, 1, 1, 1, 1, 5, 4, 4, 6, 5, 3, 4, 4, 4, 4, 1, 1, 4, 4, 1, 5, 6, 4, 4, 6, 4, 1, 1, 4, 5, 6, 4, 5, 6, 5, 6, 4, 1, 4, 4, 6, 1, 4, 4, 1, 5, 1, 6, 5, 4, 5, 4, 4, 5, 6, 5, 1, 4, 4, 4, 1, 5, 4, 4, 4, 1, 4, 4, 4, 4, 4, 1, 1, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 5, 1, 4, 4, 4, 1, 4, 6, 3, 6, 4, 4, 4, 4, 1, 5, 4, 5, 3, 3, 4, 4, 5, 5, 1, 4, 4, 4, 4, 4, 4, 1, 5, 4, 4, 4, 4, 5, 6, 5, 1, 4, 4, 5, 1, 4, 4, 4, 1, 5, 5, 4, 5, 5, 5, 4, 4, 4, 4, 4, 1, 4, 1, 5, 5, 4, 4, 4, 4, 4, 5, 1, 5, 5, 5, 4, 4, 4, 5, 1, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 4, 3, 6, 3, 1, 4, 5, 4, 4, 4, 4, 1, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 6, 5, 1, 1, 5, 4, 4, 1, 4, 4, 4, 4, 4, 

# Training classification with SVM
Loading in the test sets properly and training them for an SVM model



In [ ]:
if not skip:
  LABEL_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset/labels')

  def get_number(fileName):
   trim1 = fileName[6:]
   trim2 = trim1[:5]
   cleanId = trim2.lstrip('0')
   return int(cleanId)


  TRAIN_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset/train')
  training_data = []
  train_labels = get_train_labels() 

  def create_training_data():
    for img in os.listdir(TRAIN_FOLDER):
      try:
        img_array = cv2.imread(os.path.join(TRAIN_FOLDER,img),cv2.IMREAD_GRAYSCALE)
        get_number(img)
        training_data.append([img_array,train_labels[get_number(img)-1]])
      except Exception as e:
       print(e)
      
  
  create_training_data()

  random.shuffle(training_data)

In [ ]:

if not skip:
  feature_vector = get_feature_vector(training_data[1][0])
  print(training_data[1][0])
  print(feature_vector)
  print(feature_vector.shape)
  print(feature_vector.size)


**Convert input data to feature vectors and save into csv for future use**
I decided to try writing all of the training and test data into single csvs in their vector forms for the ease of training and testing later. All the preprocessing is done at this point so its just the raw data. 


Here are the references of code I used to create this code https://www.codegrepper.com/code-examples/python/how+to+store+numpy+array+in+csv+file 

https://stackoverflow.com/questions/61445049/pandas-to-csv-writing-keeps-consuming-more-memory-until-it-crashes 



In [ ]:
if not skip:
#writing it all in one go
  featureVectorsAndLables = []

  segmentTest = training_data[:80]


  for image, label in segmentTest:
    fV = get_feature_vector(image)
    fV = np.append(fV,[label])
    print(label)

    featureVectorsAndLables.append(fV)
    print(len(segmentTest)-len(featureVectorsAndLables))
  #reference  https://www.codegrepper.com/code-examples/python/how+to+store+numpy+array+in+csv+file

  featureVectorsAndLables = np.array(featureVectorsAndLables)
  import pandas as pd 
  pd.DataFrame(featureVectorsAndLables).to_csv(os.path.join(TRAIN_FOLDER,"train_data.csv"),index=False)


In [ ]:
if not skip:
#Split training data into chunks

#convert training data into np array
  chunk_training_data = np.array(training_data,dtype=object) 
#split into a list of chunks (these chunks are training data) 12 chunks 
  chunk_training_data = np.array_split(chunk_training_data, 12)


In [ ]:
#writing in chunks
import pandas as pd 

if not skip:

  segmentTest = training_data[:80]
  chunk_training_data = np.array(segmentTest,dtype=object) 
  chunk_training_data = np.array_split(chunk_training_data, 10)

#reference https://stackoverflow.com/questions/61445049/pandas-to-csv-writing-keeps-consuming-more-memory-until-it-crashes 

  header = True
  chunkCount = 1
  imageCount = 1
  for chunk in chunk_training_data:
      featureVectorsAndLables = []
      for image,label in chunk:
        fV = get_feature_vector(image)
        fV = np.append(fV,[label])
        featureVectorsAndLables.append(fV)
        print("Chunk: ",chunkCount," Countdown: ", len(segmentTest)-imageCount)
        imageCount += 1 

      featureVectorsAndLables = np.array(featureVectorsAndLables)
      pd.DataFrame(featureVectorsAndLables).to_csv(os.path.join(TRAIN_FOLDER,"train_data_chunks.csv"),index=False,mode='a',header=header)
      header = False
      chunkCount += 1

  #  chunk.to_csv(os.path.join(TRAIN_FOLDER,"train_data_chunks.csv"),
     #   header=header, cols=[['TIME','STUFF']], mode='a',index=False)


#featureVectorsAndLables = np.array(featureVectorsAndLables)
#pd.DataFrame(featureVectorsAndLables).to_csv(os.path.join(TRAIN_FOLDER,"train_data_chunks.csv"),index=False)


In [ ]:
TRAIN_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset/train')

tData = pd.read_csv(os.path.join(TRAIN_FOLDER,"train_data_refined.csv"))
numpyTData = tData.to_numpy()
print(numpyTData[0][len(numpyTData[0])-1]) 
print(numpyTData[79][0]) 


7.0
0.0015362979857302


In [ ]:
trainData = tData.to_numpy()

from numpy.random import shuffle
shuffle(trainData)
Label_count = [0,0,0,0,0,0,0]
for sample in trainData:
  label = int(sample[len(sample)-1])-1
  Label_count[label] = Label_count[label] +1
 
print(Label_count)

train_labels = get_train_labels()

Label_count2 = [0,0,0,0,0,0,0]
for label in train_labels:
  label = int(label)-1
  Label_count2[label] = Label_count2[label] +1

print(Label_count2)





[1290, 281, 717, 4772, 1982, 705, 2524]
[1290, 281, 717, 4772, 1982, 705, 2524]


In [ ]:
#length of training data
print(trainData.shape) 

print(trainData[0])
#reformat input to data array, label array 
def formatTraining(trainingData):
  labels = []
  formattedTraining = []
  for sample in trainingData:
    labels.append(int(sample[len(sample)-1]))
    sample = sample[:-1]; sample 
    formattedTraining.append(sample) 


  return formattedTraining, labels

from numpy.random import shuffle
shuffle(trainData)

x,y = formatTraining(trainData)


X = x[:9000]
Y = y[:9000]
print(len(X))
print(X[0])
print(y)

(12271, 8501)
[ 0.16308618  0.23765052  0.2198656  ... 15.         15.
  7.        ]
9000
[0.59960756 0.23576635 0.         ... 0.         0.         0.        ]
[4, 4, 4, 4, 4, 7, 1, 3, 4, 6, 4, 6, 6, 1, 3, 5, 4, 3, 4, 3, 4, 5, 7, 7, 4, 6, 4, 7, 4, 4, 5, 2, 3, 7, 5, 6, 4, 7, 7, 5, 4, 5, 3, 4, 5, 6, 5, 4, 4, 1, 7, 5, 4, 1, 4, 6, 7, 4, 4, 4, 4, 7, 4, 4, 5, 4, 4, 4, 4, 7, 7, 4, 4, 4, 4, 7, 3, 1, 4, 6, 1, 4, 3, 7, 1, 4, 5, 4, 6, 4, 4, 4, 1, 7, 5, 1, 4, 4, 4, 4, 1, 4, 5, 4, 4, 4, 4, 7, 2, 7, 4, 4, 5, 5, 4, 7, 6, 4, 5, 1, 4, 1, 7, 4, 3, 7, 4, 7, 6, 4, 3, 2, 4, 7, 4, 1, 4, 6, 4, 1, 4, 4, 4, 4, 4, 4, 4, 5, 4, 3, 4, 1, 4, 2, 7, 7, 4, 5, 4, 5, 4, 6, 3, 4, 3, 5, 4, 1, 4, 5, 4, 1, 4, 4, 4, 4, 7, 1, 1, 5, 7, 7, 4, 4, 4, 3, 4, 5, 5, 7, 7, 4, 5, 7, 1, 4, 1, 7, 1, 4, 4, 5, 4, 6, 4, 5, 4, 5, 7, 7, 4, 7, 5, 3, 4, 5, 6, 3, 4, 5, 6, 6, 4, 5, 7, 5, 7, 5, 4, 2, 4, 7, 4, 4, 7, 1, 3, 2, 6, 4, 1, 3, 4, 4, 4, 4, 4, 4, 7, 4, 5, 4, 4, 5, 6, 1, 1, 3, 1, 4, 3, 7, 4, 2, 4, 7, 5, 4, 4, 7, 5, 4, 3, 4, 4, 1, 5, 5, 4, 

# SVM TRAINING
Now the training data is formatted into a way it can be easily used we can split the data into the training and test for the model. A split of 80 - 20 is usually a good place to start. I thought I'd try a variety of different SVM methods to see which suited my dataset best.



I tested out a variety of different methods for svm here are the references for the code below https://www.baeldung.com/cs/svm-multiclass-classification


https://datascience.stackexchange.com/questions/57863/why-my-svm-svc-fit-linear-kernal-run-so-long-time 


https://archive.siam.org/meetings/sdm01/pdf/sdm01_13.pdf 

https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/ 

https://medium.com/analytics-vidhya/hyperparameter-tuning-an-svm-a-demonstration-using-hyperparameter-tuning-cross-validation-on-96b05db54e5b




In [ ]:
from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


#svm references https://www.baeldung.com/cs/svm-multiclass-classification

#reference https://datascience.stackexchange.com/questions/57863/why-my-svm-svc-fit-linear-kernal-run-so-long-time 


#reference https://archive.siam.org/meetings/sdm01/pdf/sdm01_13.pdf 

#reference https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/ 

#reference https://medium.com/analytics-vidhya/hyperparameter-tuning-an-svm-a-demonstration-using-hyperparameter-tuning-cross-validation-on-96b05db54e5b

if not skip: 
  X_train, X_test, y_train, y_test = model_selection.train_test_split(X,Y, train_size=0.80, test_size=0.20)

  poly = svm.SVC(kernel='poly', degree=3, C=1,verbose=True).fit(X_train, y_train)
  poly_pred = poly.predict(X_test)
  poly_accuracy = accuracy_score(y_test, poly_pred)
  poly_f1 = f1_score(y_test, poly_pred, average='weighted')
  print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
  print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

In [ ]:
print(y)

[4, 4, 4, 4, 4, 7, 1, 3, 4, 6, 4, 6, 6, 1, 3, 5, 4, 3, 4, 3, 4, 5, 7, 7, 4, 6, 4, 7, 4, 4, 5, 2, 3, 7, 5, 6, 4, 7, 7, 5, 4, 5, 3, 4, 5, 6, 5, 4, 4, 1, 7, 5, 4, 1, 4, 6, 7, 4, 4, 4, 4, 7, 4, 4, 5, 4, 4, 4, 4, 7, 7, 4, 4, 4, 4, 7, 3, 1, 4, 6, 1, 4, 3, 7, 1, 4, 5, 4, 6, 4, 4, 4, 1, 7, 5, 1, 4, 4, 4, 4, 1, 4, 5, 4, 4, 4, 4, 7, 2, 7, 4, 4, 5, 5, 4, 7, 6, 4, 5, 1, 4, 1, 7, 4, 3, 7, 4, 7, 6, 4, 3, 2, 4, 7, 4, 1, 4, 6, 4, 1, 4, 4, 4, 4, 4, 4, 4, 5, 4, 3, 4, 1, 4, 2, 7, 7, 4, 5, 4, 5, 4, 6, 3, 4, 3, 5, 4, 1, 4, 5, 4, 1, 4, 4, 4, 4, 7, 1, 1, 5, 7, 7, 4, 4, 4, 3, 4, 5, 5, 7, 7, 4, 5, 7, 1, 4, 1, 7, 1, 4, 4, 5, 4, 6, 4, 5, 4, 5, 7, 7, 4, 7, 5, 3, 4, 5, 6, 3, 4, 5, 6, 6, 4, 5, 7, 5, 7, 5, 4, 2, 4, 7, 4, 4, 7, 1, 3, 2, 6, 4, 1, 3, 4, 4, 4, 4, 4, 4, 7, 4, 5, 4, 4, 5, 6, 1, 1, 3, 1, 4, 3, 7, 4, 2, 4, 7, 5, 4, 4, 7, 5, 4, 3, 4, 4, 1, 5, 5, 4, 6, 4, 5, 3, 4, 3, 6, 3, 4, 7, 4, 3, 4, 4, 7, 4, 7, 1, 7, 4, 7, 4, 7, 6, 7, 4, 3, 7, 7, 7, 3, 1, 4, 5, 4, 4, 4, 1, 6, 7, 5, 7, 7, 4, 7, 4, 4, 1, 1, 4, 7, 5, 4, 4, 

In [ ]:
from sklearn.preprocessing import StandardScaler

if skip:
  scaler = StandardScaler()
  x = scaler.fit_transform(x)
  X_train, X_test, y_train, y_test = model_selection.train_test_split(x,y, train_size=0.90, test_size=0.10)
  

  X_train = np.array(X_train)
  X_test= np.array(X_test)
  y_train= np.array(y_train)
  y_test = np.array(y_test)

  from sklearn.svm import LinearSVC
  clf = LinearSVC(random_state=1, tol=1e-5,max_iter=10000,multi_class="ovr",verbose=1,dual=False, class_weight= "balanced",C=10)
  clf.fit(X_train, y_train.ravel()) 
  clf_pred = clf.predict(X_test)
  clf_accuracy = accuracy_score(y_test, clf_pred)
  clf_f1 = f1_score(y_test, clf_pred, average='weighted')
  print('Accuracy (Linear Kernel): ', "%.2f" % (clf_accuracy*100))
  print('F1 (Linear Kernel): ', "%.2f" % (clf_f1*100))

[LibLinear]

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC




if not skip:
  #reference https://medium.com/analytics-vidhya/hyperparameter-tuning-an-svm-a-demonstration-using-hyperparameter-tuning-cross-validation-on-96b05db54e5b

    X_train, X_test, y_train, y_test = model_selection.train_test_split(x,y, train_size=0.80, test_size=0.20)
    n_estimators = 10
    clf = OneVsRestClassifier(BaggingClassifier(LinearSVC(random_state=2, tol=1e-5,max_iter = 10000), max_samples=1.0 / n_estimators, n_estimators=n_estimators))
    clf.fit(X_train, y_train)
    clf_pred = clf.predict(X_test)
    clf_accuracy = accuracy_score(y_test, clf_pred)
    clf_f1 = f1_score(y_test, clf_pred, average='weighted')
    print('Accuracy (Ensemble SVC): ', "%.2f" % (clf_accuracy*100))
    print('F1 (Ensemble SVC): ', "%.2f" % (clf_f1*100))
